<a href="https://colab.research.google.com/github/torikabuto-fkg/-_Anki-/blob/main/QB_csv_Ver_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #ライブラリのインストール
#@markdown [shift]+[Enter]を押して起動してください

!pip install PyMuPDF==1.21.1
!pip install pdfminer.six==20221105
!pip install PyPDF2==3.0.1
!pip install natsort==5.5.0
#正規表現
import re
import io, copy
import glob, os
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox, LTTextLine, LTChar
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage
import PyPDF2
from PyPDF2 import PdfFileWriter, PdfFileReader
import fitz
from typing import Tuple
import os
import sys
import shutil
import csv
import pandas as pd
import time
from natsort import natsorted
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: natsort
    Found existing installation: natsort 8.4.0
    Uninstalling natsort-8.4.0:
      Successfully uninstalled natsort-8.4.0


In [61]:
#@title #グーグルドライブと連携
#@markdown [shift]+[Enter]を押して起動してください
#@markdown　<br>GogoleDriveとの連携が求められるので、許可してください。

#ドライブ設定
PATH_GMOUNT='/content/gdrive'
PATH_MYDRIVE=PATH_GMOUNT+'/My Drive'

# Check if the mount point directory exists and is empty
import os
import shutil

# Instead of removing the entire mount point, only remove the folder_name directory if it exists
#if os.path.exists(os.path.join(PATH_MYDRIVE, folder_name)): # This line was causing the issue, since it check if something exists in My Drive before it is mounted
if os.path.exists(PATH_GMOUNT):   # Check if the mount point exists first
  if os.listdir(PATH_GMOUNT):  # Check if the mount point is empty
    print(f"Existing directory '{PATH_GMOUNT}' is not empty and will be cleared for mounting.")  # This should be clear to the user that we are going to delete potentially usefull data.
    shutil.rmtree(PATH_GMOUNT)  # Remove everything within the mount directory, if it is not empty
    print(f"Directory '{PATH_GMOUNT}' cleared.")

#GDriveマウント
from google.colab import drive
drive.mount(PATH_GMOUNT)



Mounted at /content/gdrive


In [109]:
import pandas as pd
import re
import os

# CSVファイルの読み込み
path = "4B-03"
file_path = f"/content/{path}.csv"  # 適宜ファイルパスを変更してください
df = pd.read_csv(file_path)

# 選択肢を分類するための辞書
choices_dict = {'a': [], 'b': [], 'c': [], 'd': [], 'e': []}

# 新しい「問題」列を格納するリスト
processed_questions = []

# 各行の「問題」列から選択肢を抽出し、元のテキストから削除
for question in df["問題"]:
    # 選択肢を正規表現で抽出（例: "ａ 核", "ｂ リソソーム" など）
    matches = re.findall(r'(ａ|ｂ|ｃ|ｄ|ｅ)\s*([^\n]+)', str(question))

    # 各選択肢を対応するリストに格納
    extracted = {'a': '', 'b': '', 'c': '', 'd': '', 'e': ''}
    for match in matches:
        key = match[0]  # "ａ", "ｂ", ... "ｅ"
        value = match[1]  # 選択肢の内容
        extracted[key.replace('ａ', 'a').replace('ｂ', 'b').replace('ｃ', 'c').replace('ｄ', 'd').replace('ｅ', 'e')] = value

    # 選択肢を削除した問題文を作成
    cleaned_question = re.sub(r'(ａ|ｂ|ｃ|ｄ|ｅ)\s*[^\n]+', '', str(question)).strip()
    processed_questions.append(cleaned_question)

    # 各リストに追加
    for key in choices_dict.keys():
        choices_dict[key].append(extracted[key])

# 抽出したデータを新しいDataFrameに追加
df_processed = df.copy()
df_processed["問題"] = processed_questions
for key in choices_dict.keys():
    df_processed[f"{key}選択肢"] = choices_dict[key]

# 処理したデータをCSVとして保存
output_file = f"/content/Anki/{path}/{path}.csv"

# ディレクトリが存在しない場合は作成
os.makedirs(os.path.dirname(output_file), exist_ok=True)

df_processed.to_csv(output_file, index=False)

print(f"処理が完了しました。結果は {output_file} に保存されました。")


処理が完了しました。結果は /content/Anki/4B-03/4B-03.csv に保存されました。


In [110]:
import shutil

# /content/ をZIPに圧縮
shutil.make_archive('/content/my_content', 'zip', '/content', 'Anki/')

# ZIPファイルをダウンロード
files.download('/content/my_content.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>